In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
import os.path
import pandas
from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from  mefs_from_simple_dispatch import generateMefs
from  mefs_from_simple_dispatch import plotDispatch
from simple_dispatch import generatorDataShort

In [57]:
run_year = 2019
egrid_data_xlsx = 'egrid2019_data.xlsx'
nerc_region = 'WECC'

In [24]:
gd_short = pickle.load(open('generator_data_short_%s_%s.obj'%(nerc_region, str(run_year)), 'rb'))

# 4. Additions

https://www.wecc.org/Administrative/06_Brown_Retirements_October%202020.pdf

In [58]:
# gd_short.df.head()
egrid_unt = pandas.read_excel(egrid_data_xlsx, 'UNT'+'19', skiprows=[0])
egrid_unt['orispl_unit'] = egrid_unt.ORISPL.map(str) + '_' + egrid_unt.UNITID.map(str)
egrid_gen = pandas.read_excel(egrid_data_xlsx, 'GEN'+'19', skiprows=[0])
egrid_plnt = pandas.read_excel(egrid_data_xlsx, 'PLNT'+'19', skiprows=[0])
# unit_drops.to_csv('scheduled_retirements_2019.csv')

In [59]:
unit_drops = pd.read_csv('scheduled_retirements_2019.csv', index_col=0)
unit_drops.head()


,WECC Name,orispl,orispl_unit,retirement_year
0,H Wilson Sundt Generating Station,126.0,126_1,2020.0
1,H Wilson Sundt Generating Station,126.0,126_2,2020.0
2,Colstrip,6076.0,6076_1,2020.0
3,Colstrip,6076.0,6076_2,2020.0
4,Ontario Mill,10427.0,10427_GEN1,2020.0


In [60]:
egrid_gen['orispl_unit'] = egrid_gen['ORISPL'].map(str) + '_' + egrid_gen['GENID'].map(str)

In [61]:
leaving_gens = egrid_gen[egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values)].loc[:, ['PSTATABB', 'PNAME', 'GENID', 'FUELG1', 'NAMEPCAP', 'GENYRRET', 'orispl_unit']].copy(deep=True)
leaving_gens.head()

,PSTATABB,PNAME,GENID,FUELG1,NAMEPCAP,GENYRRET,orispl_unit
1377,AZ,Cholla,1,SUB,113.6,NaN,113_1
1378,AZ,Cholla,3,SUB,312.3,NaN,113_3
1379,AZ,Cholla,4,SUB,414.0,2020.0,113_4
1593,AZ,Ocotillo,1,NG,113.6,2019.0,116_1
1594,AZ,Ocotillo,2,NG,113.6,2019.0,116_2


In [62]:
egrid_gen.head()

,SEQGEN19,YEAR,PSTATABB,PNAME,ORISPL,GENID,NUMBLR,GENSTAT,PRMVR,FUELG1,NAMEPCAP,CFACT,GENNTAN,GENNTOZ,GENERSRC,GENYRONL,GENYRRET,orispl_unit
0,1,2019,AK,7-Mile Ridge Wind Project,60814,WT1,0,CN,WT,WND,1.8,NaN,NaN,NaN,NaN,NaN,NaN,60814_WT1
1,2,2019,AK,Agrium Kenai Nitrogen Operations,54452,744A,0,OS,GT,NG,2.5,NaN,NaN,NaN,NaN,1977.0,NaN,54452_744A
2,3,2019,AK,Agrium Kenai Nitrogen Operations,54452,744B,0,OS,GT,NG,2.5,NaN,NaN,NaN,NaN,1977.0,NaN,54452_744B
3,4,2019,AK,Agrium Kenai Nitrogen Operations,54452,744C,0,OS,GT,NG,2.5,NaN,NaN,NaN,NaN,1977.0,NaN,54452_744C
4,5,2019,AK,Agrium Kenai Nitrogen Operations,54452,744D,0,OS,GT,NG,2.5,NaN,NaN,NaN,NaN,1977.0,NaN,54452_744D


In [63]:
staying_gens = egrid_gen.loc[(egrid_gen['orispl_unit'].isin(gd_short.df['orispl_unit'].values))&(~(egrid_gen['orispl_unit'].isin(leaving_gens['orispl_unit'].values)))].loc[:, ['PSTATABB', 'PNAME', 'GENID', 'FUELG1', 'NAMEPCAP', 'PRMVR', 'GENYRONL', 'GENYRRET', 'orispl_unit', 'ORISPL']].copy(deep=True)
staying_gens.head()

,PSTATABB,PNAME,GENID,FUELG1,NAMEPCAP,PRMVR,GENYRONL,GENYRRET,orispl_unit,ORISPL
1331,AZ,Arlington Valley Energy Facility,CTG1,NG,198.0,CT,2002.0,NaN,55282_CTG1,55282
1332,AZ,Arlington Valley Energy Facility,CTG2,NG,198.0,CT,2002.0,NaN,55282_CTG2,55282
1358,AZ,Black Mountain Generating Station,1,NG,60.5,GT,2008.0,NaN,56482_1,56482
1359,AZ,Black Mountain Generating Station,2,NG,60.5,GT,2008.0,NaN,56482_2,56482
1481,AZ,H Wilson Sundt Generating Station,4,NG,173.3,ST,1967.0,NaN,126_4,126


In [64]:
staying_gens = staying_gens.sort_values(by=['PSTATABB', 'FUELG1', 'PRMVR', 'NAMEPCAP'], ascending=[True, True, True, False])

In [65]:
staying_gens.head()

,PSTATABB,PNAME,GENID,FUELG1,NAMEPCAP,PRMVR,GENYRONL,GENYRRET,orispl_unit,ORISPL
1660,AZ,South Point Energy Center,A,NG,236.0,CT,2001.0,NaN,55177_A,55177
1661,AZ,South Point Energy Center,B,NG,236.0,CT,2001.0,NaN,55177_B,55177
1331,AZ,Arlington Valley Energy Facility,CTG1,NG,198.0,CT,2002.0,NaN,55282_CTG1,55282
1332,AZ,Arlington Valley Energy Facility,CTG2,NG,198.0,CT,2002.0,NaN,55282_CTG2,55282
1588,AZ,Ocotillo,GT3,NG,161.9,GT,2019.0,NaN,116_GT3,116


In [18]:
totals = staying_gens.groupby(['ORISPL'], as_index=False).sum()
for i in totals.index:
    totals.loc[i, 'FUELG1'] = staying_gens.loc[staying_gens[staying_gens['ORISPL']==totals.loc[i, 'ORISPL']].index, 'FUELG1'].values[0]
    totals.loc[i, 'PSTATABB'] = staying_gens.loc[staying_gens[staying_gens['ORISPL']==totals.loc[i, 'ORISPL']].index, 'PSTATABB'].values[0]

In [21]:
states_subsets = {'PNW':['WA', 'OR', 'ID', 'MT', 'UT', 'NV'], 'RM':['CO', 'WY'], 'DSW':['AZ', 'NM'], 'CA':['CA']}
# ct: combined cycle (newest), gt: combustion turbine, st: old

Use 'MW' to judge capacity

In [85]:
gd_short.df[gd_short.df.orispl.isin(staying_gens.ORISPL)]

,ba,co21,co210,co211,co212,co213,co214,co215,co216,co217,...,fuel_price45,fuel_price46,fuel_price47,fuel_price48,fuel_price49,fuel_price50,fuel_price51,fuel_price52,min_out_multiplier,min_out
1,CISO,575.900478,596.150143,599.390089,599.390089,599.390089,599.390089,599.390089,599.390089,599.390089,...,2.49124,2.49124,2.49124,2.77522,2.77522,2.77522,2.77522,2.77522,0.5,19.474673
3,CISO,546.510324,559.862743,643.276636,502.278095,491.061008,469.410508,616.815110,467.767266,577.299545,...,2.923,2.923,2.923,3.985,3.985,3.985,3.985,3.985,0.5,21.627335
4,CISO,439.170204,439.170204,439.170204,439.170204,439.170204,439.170204,439.170204,439.170204,439.170204,...,2.923,2.923,2.923,3.985,3.985,3.985,3.985,3.985,0.5,55.218727
10,PSCO,814.721913,793.055274,812.977327,812.977327,800.266768,800.266768,800.266768,800.266768,816.466500,...,2.6307,2.6307,2.6307,3.5865,3.5865,3.5865,3.5865,3.5865,0.5,16.000000
11,PSCO,538.974618,543.471014,558.580102,558.580102,559.716028,559.716028,562.824980,562.824980,562.824980,...,2.6307,2.6307,2.6307,3.5865,3.5865,3.5865,3.5865,3.5865,0.5,33.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,PACE,936.347293,894.897189,918.188010,923.466307,944.095138,922.914175,918.041360,951.307180,944.188599,...,1.96114,1.96114,1.96114,1.96962,1.96962,1.96962,1.96962,1.96962,0.4,194.000000
585,TEPC,940.041056,945.369936,950.126516,913.402467,914.815228,918.524596,922.068583,911.212253,900.953354,...,1.93139,1.93139,1.93139,1.96722,1.96722,1.96722,1.96722,1.96722,0.4,170.000000
586,TEPC,856.585464,862.330148,855.773397,841.417235,841.417235,841.417235,908.586058,857.436145,859.933308,...,1.93139,1.93139,1.93139,1.96722,1.96722,1.96722,1.96722,1.96722,0.4,170.000000
587,TEPC,924.646304,911.061567,921.400265,930.636161,937.560414,934.277403,934.277403,934.277403,934.277403,...,1.93139,1.93139,1.93139,1.96722,1.96722,1.96722,1.96722,1.96722,0.4,179.600000


In [96]:
staying_gens = staying_gens.reset_index(drop=True)
for i in staying_gens.index:
    idx = gd_short.df[gd_short.df.orispl_unit==staying_gens.loc[i, 'orispl_unit']].index
    staying_gens.loc[i, 'gd_mw'] = gd_short.df.loc[idx, 'mw'].values[0]
    staying_gens.loc[i, 'gd_max_mwh'] = gd_short.df.loc[idx, ['mw'+str(i) for i in np.arange(1, 53)]].max(axis=1).values[0]

In [97]:
staying_gens

,PSTATABB,PNAME,GENID,FUELG1,NAMEPCAP,PRMVR,GENYRONL,GENYRRET,orispl_unit,ORISPL,gd_mw,gd_max_mwh
0,AZ,South Point Energy Center,A,NG,236.0,CT,2001.0,NaN,55177_A,55177,254.0,247.0
1,AZ,South Point Energy Center,B,NG,236.0,CT,2001.0,NaN,55177_B,55177,252.0,250.0
2,AZ,Arlington Valley Energy Facility,CTG1,NG,198.0,CT,2002.0,NaN,55282_CTG1,55282,298.0,295.0
3,AZ,Arlington Valley Energy Facility,CTG2,NG,198.0,CT,2002.0,NaN,55282_CTG2,55282,304.0,303.0
4,AZ,Ocotillo,GT3,NG,161.9,GT,2019.0,NaN,116_GT3,116,109.0,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...
185,WY,Laramie River Station,1,RC,570.0,ST,1981.0,NaN,6204_1,6204,614.0,612.0
186,WY,Laramie River Station,2,RC,570.0,ST,1981.0,NaN,6204_2,6204,616.0,616.0
187,WY,Laramie River Station,3,RC,570.0,ST,1982.0,NaN,6204_3,6204,613.0,613.0
188,WY,Dry Fork Station,01,SUB,483.7,ST,2011.0,NaN,56609_01,56609,453.0,451.0


Use gen data for the fuel type and year online and prmvr, use gd_short for the mw and mwh capacity.

In [99]:
# Additions: 
additions_df = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
states_subsets = {'PNW':['WA', 'OR', 'ID', 'MT', 'UT', 'NV'], 'RM':['CO', 'WY'], 'DSW':['AZ', 'NM'], 'CA':['CA']}

In [116]:
# Gas 1340.5 PNW 2021
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['55700_CTG1', '2336_8', '2336_9', '56237_CT01', '3648_4', '3648_5', '3648_6', '55544_GT1']):
    test.loc[i, 'Year'] = 2021; test.loc[i, 'Fuel Type'] = 'NG'; test.loc[i, 'PRMVR'] = 'CT'; test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
additions_df = pd.concat((additions_df, test), ignore_index=True)

1347.0


In [118]:
diff = 1340.5 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()

In [121]:
# Gas 130 DSW 2021
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['116_GT3']):
    test.loc[i, 'Year'] = 2021; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'DSW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 130 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

109.0


In [137]:
# Gas 1338 CA/MX 2021
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['422_GT5', '404_4']):
    test.loc[i, 'Year'] = 2021; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 1338 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['57555_CTG1', '57555_CTG2', '57555_CTG3', '404_7', '404_6', '57267_1', '57267_2']):
    test.loc[i, 'Year'] = 2021; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
    
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

386.0
959.0


In [141]:
# Gas 86 RM 2021
test = pd.DataFrame({'Year':[2021], 'Fuel Type':['NG'], 'PRMVR':['GT'], 'Large Region':['RM'], 'orispl_unit':['6761_C']})
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 86 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

85.0


In [153]:
# Gas 1102 PNW 2022
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['2336_8', '56237_CT01', '56237_CT02', '55662_CT1']):
    test.loc[i, 'Year'] = 2022; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 1102 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

1094.0


In [159]:
# Gas 1580 PNW 2023
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['55700_CTG1', '2336_8', '2336_9', '56237_CT01', '56237_CT02']):
    test.loc[i, 'Year'] = 2023; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 1580 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['3648_4', '3648_5', '3648_6']):
    test.loc[i, 'Year'] = 2023; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

1461.0
125.0


In [167]:
# Gas 838 DSW 2023
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['116_GT3', '116_GT4', '116_GT5', '116_GT6', '116_GT7', '2444_9', '55802_CTG1', '56482_1']):
    test.loc[i, 'Year'] = 2023; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'DSW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 838 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

849.0


In [177]:
# Gas 637 CA/MX 2023
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['404_4']):
    test.loc[i, 'Year'] = 2023; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 637 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['57555_CTG1','404_6', '404_7']):
    test.loc[i, 'Year'] = 2023; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

317.0
323.0


In [182]:
# Gas 300 CA/MX 2024
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['422_GT5', '57555_CTG1', '57555_CTG2']):
    test.loc[i, 'Year'] = 2024; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
test.loc[0, 'PRMVR'] = 'CT'
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 300 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


297.0


In [189]:
# Gas 571 PNW 2024
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['2336_9', '56237_CT01']):
    test.loc[i, 'Year'] = 2024; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 571 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


567.0


In [192]:
# Gas 45 DSW 2024
test = pd.DataFrame({'Year':[2024], 'Fuel Type':['NG'], 'PRMVR':['GT'], 'Large Region':['DSW'], 'orispl_unit':['56482_1']})
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 45 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


47.0


In [201]:
# Gas 844 RM 2025
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['469_5', '469_6']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'RM'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 844 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['6112_5', '56998_CT08']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'RM'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

633.0
204.0


In [205]:
# Gas 277.5 PNW 2025
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['56237_CT01']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 277.5 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)



265.0


In [209]:
# Gas 200 DSW 2025
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['116_GT3', '116_GT4']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'DSW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 200 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


219.0


In [215]:
# Gas 90 CA/MX 2025
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['56232_2', '55538_CTG1']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 90 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


95.0


In [224]:
# Gas 232.5 PNW 2026
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['3648_4', '3648_5', '3648_6', '55622_U1','55622_U2', '55544_GT1']):
    test.loc[i, 'Year'] = 2025; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 232.5 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


234.0


In [240]:
# Gas 400 DSW 2026
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['116_GT6', '116_GT3', '56482_1', '56482_2', '2444_9']):
    test.loc[i, 'Year'] = 2026; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'DSW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 400 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)



401.0


In [243]:
# Gas 185 RM 2026
test = pd.DataFrame({'Year':[2026], 'Fuel Type':['NG'], 'PRMVR':['GT'], 'Large Region':['RM'], 'orispl_unit':['6112_5']})
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 185 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


163.0


In [246]:
# Gas 45 PNW 2027
test = pd.DataFrame({'Year':[2027, 2027], 'Fuel Type':['NG', 'NG'], 'PRMVR':['GT', 'GT'], 'Large Region':['PNW', 'PNW'], 'orispl_unit':['55544_GT1', '55544_GT2']})
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 45 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


52.0


In [252]:
# Gas 320 DSW 2027
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['116_GT3', '116_GT4', '116_GT5']):
    test.loc[i, 'Year'] = 2027; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'DSW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 320 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


328.0


In [254]:
# Gas 50 DSW 2028
test = pd.DataFrame({'Year':[2028], 'Fuel Type':['NG'], 'PRMVR':['GT'], 'Large Region':['DSW'], 'orispl_unit':['56482_1']})
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 50 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)



47.0


In [260]:
# Gas 800 PNW 2028
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['56237_CT01', '56237_CT02', '55662_CT1']):
    test.loc[i, 'Year'] = 2028; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 800 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


794.0


In [271]:
# Gas 1234 CA/MX 2029
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['422_GT5', '404_4']):
    test.loc[i, 'Year'] = 2029; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 1234 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['57555_CTG1', '57555_CTG2', '57555_CTG3', '404_6', '57267_1', '57267_2']):
    test.loc[i, 'Year'] = 2029; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'CA/MX'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


386.0
854.0


In [279]:
# Gas 700 PNW 2029
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['55700_CTG1', '2336_8']):
    test.loc[i, 'Year'] = 2029; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 700 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['3648_4', '3648_5']):
    test.loc[i, 'Year'] = 2029; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 0 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


629.0
84.0


In [283]:
# Gas 569 PNW 2030
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['2336_8', '56237_CT01']):
    test.loc[i, 'Year'] = 2030; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'CT'; 
    test.loc[i, 'Large Region'] = 'PNW'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 569 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)


565.0


In [290]:
# Gas 370 RM 2030
test = pd.DataFrame({'Year':[], 'Fuel Type':[], 'PRMVR':[], 'Large Region':[], 'orispl_unit':[]})
for i, val in enumerate(['6112_5', '6112_6', '56998_CT08']):
    test.loc[i, 'Year'] = 2030; test.loc[i, 'Fuel Type'] = 'NG'; 
    test.loc[i, 'PRMVR'] = 'GT'; 
    test.loc[i, 'Large Region'] = 'RM'
    test.loc[i, 'orispl_unit'] = val
print(staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum())
diff += 370 - staying_gens[staying_gens['orispl_unit'].isin(test['orispl_unit'].values)]['gd_mw'].sum()
additions_df = pd.concat((additions_df, test), ignore_index=True)

364.0


In [291]:
diff

1.5

In [293]:
additions_df

,Year,Fuel Type,PRMVR,Large Region,orispl_unit
0,2021.0,NG,CT,PNW,55700_CTG1
1,2021.0,NG,CT,PNW,2336_8
2,2021.0,NG,CT,PNW,2336_9
3,2021.0,NG,CT,PNW,56237_CT01
4,2021.0,NG,CT,PNW,3648_4
...,...,...,...,...,...
91,2030.0,NG,CT,PNW,2336_8
92,2030.0,NG,CT,PNW,56237_CT01
93,2030.0,NG,GT,RM,6112_5
94,2030.0,NG,GT,RM,6112_6


In [292]:
# old_additions = pd.read_csv('generator_additions.csv', index_col=0)
# old_additions.to_csv('generator_additions_old.csv')
# additions_df.to_csv('generator_additions.csv')